In [1]:
from langame import LangameClient
import json
import openai
import datetime
from langame.conversation_starters import get_existing_conversation_starters
import logging
import os
from datasets import load_dataset
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, AutoTokenizer, DataCollatorForLanguageModeling, DataCollatorForLanguageModeling

c = LangameClient("../config.yaml")

# topic classification

In [ ]:
out_file_name = f"../data/fine_tune_topic_classification_{datetime.date.today().strftime('%d_%m_%Y')}.jsonl"

with open(out_file_name, 'w') as outfile:
    for e in c._firestore_client.collection("memes").stream():
        json.dump({
            "text": e.get("content"),
            "label": e.get("topics")[0]
        }, outfile)
        outfile.write('\n')

In [ ]:
c_f = openai.File.create(
    file=open(file_name),
    purpose="classifications"
)

In [ ]:
openai.Classification.create(
    file="file-rV9rZbiiziXLfjqc4MKLLChy",
    query="123",
    search_model="ada", 
    model="curie", 
    max_examples=3
)

# quality classification

In [2]:
from langame.conversation_starters import get_existing_conversation_starters
import logging
logger = logging.getLogger("classification")
memes, index, embeddings_model = get_existing_conversation_starters(
    c._firestore_client, logger=logger, confirmed=True)

Skipping id: 4XlXSjPhH6QSGZDzwIQe, garbage
Skipping id: CYsOWtp1ed2XDeplB5O5, garbage
Skipping id: DlXglThZUiPJcuyLLKHV, garbage
Skipping id: NtXdp7ODn4sR3YR2eEW6, garbage
Skipping id: U0DUXOSpfqYsgurOyL3L, garbage
Skipping id: Uikgs6eO85MRivL0peL6, garbage
Skipping id: XIPyRwJloK6ZdAdTHOeu, garbage
Skipping id: d9FxG5haHnN2ku7spAng, garbage
Skipping id: eyzSmD1AdfKkwOYtfoof, garbage
Skipping id: f1ojlkGYgnOI4MLRGC4T, garbage
Skipping id: fySI8xbjfC5hx7QdHpva, garbage
Skipping id: gLDxkmHFL3OFufeje6E8, garbage
Skipping id: jkkOtT3PVoKQMmpA947Y, garbage
Skipping id: l1HjGNlNwpFzTnCsJxJA, garbage
Skipping id: o7jTZJyRoBxaDxIQwyQg, garbage
Skipping id: osfrRcogOlIrEtxp5fpI, garbage
Skipping id: pXXGv1o80FIldPiytsN7, garbage
Skipping id: pf1tiPMceCwvXLUfKQwY, garbage
Skipping id: q2CEZCITA0RVSsfTQRP0, garbage
Skipping id: qAwPXAX5efpoQt0TLoUu, garbage
Skipping id: qXEJ0wGr86cw8CXIgL4z, garbage
Skipping id: r5vxgYs9FnZYOxrh2tcj, garbage
Skipping id: rdguN3u8mWRuiRXfUKM8, garbage
Skipping id

Using 8 omp threads (processes), consider increasing --nb_cores if you have more
Launching the whole pipeline 05/01/2022, 10:25:03
There are 2052 embeddings of dim 768
	Compute estimated construction time of the index 05/01/2022, 10:25:03
		-> Train: 16.7 minutes
		-> Add: 0.0 seconds
		Total: 16.7 minutes
	>>> Finished "Compute estimated construction time of the index" in 0.0001 secs
	Checking that your have enough memory available to create the index 05/01/2022, 10:25:03
7.5MB of memory will be needed to build the index (more might be used if you have more)
	>>> Finished "Checking that your have enough memory available to create the index" in 0.0004 secs
	Selecting most promising index types given data characteristics 05/01/2022, 10:25:03
	>>> Finished "Selecting most promising index types given data characteristics" in 0.0000 secs
	Creating the index 05/01/2022, 10:25:03
		-> Instanciate the index HNSW15 05/01/2022, 10:25:03
		>>> Finished "-> Instanciate the index HNSW15" in 0.0034

100%|██████████| 1/1 [00:00<00:00, 273.01it/s]

		>>> Finished "-> Adding the vectors to the index" in 0.0805 secs
	>>> Finished "Creating the index" in 0.0856 secs
	Computing best hyperparameters 05/01/2022, 10:25:03


	>>> Finished "Computing best hyperparameters" in 2.8843 secs
The best hyperparameters are: efSearch=16383
	Compute fast metrics 05/01/2022, 10:25:06
2000
	>>> Finished "Compute fast metrics" in 6.9442 secs
	Saving the index on local disk 05/01/2022, 10:25:13
	>>> Finished "Saving the index on local disk" in 0.0144 secs
Recap:
{'99p_search_speed_ms': 4.525868840003113,
 'avg_search_speed_ms': 3.4529195985000882,
 'compression ratio': 0.9576115129509153,
 'index_key': 'HNSW15',
 'index_param': 'efSearch=16383',
 'nb vectors': 2052,
 'reconstruction error %': 0.0,
 'size in bytes': 6582778,
 'vectors dimension': 768}
>>> Finished "Launching the whole pipeline" in 10.2528 secs


In [3]:
out_file_name = f"../data/fine_tune_fitness_classification_{datetime.date.today().strftime('%d_%m_%Y')}.jsonl"

for e in memes:
    with open(out_file_name, "a+") as outfile:
        json.dump({
            "prompt": f"{','.join(e['topics'])} ### {e['content']} ~~~",
            "completion": f" 1\n",
        }, outfile)
        outfile.write('\n')
for e in c._firestore_client.collection("deleted_memes").stream():
    # check has "topics" and "content"
    if "topics" not in e.to_dict() or "content" not in e.to_dict():
        continue
    with open(out_file_name, "a+") as outfile:
        json.dump({
            "prompt": f"{','.join(e.get('topics'))} ### {e.get('content')} ~~~",
            "completion": f" 0\n",
        }, outfile)
        outfile.write('\n')

In [7]:
!head -5 $out_file_name | jq .

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "prompt": "ice breaker ### When is a time when you know for sure you'll soon have to exchange a nice, meaningful conversation with someone new? ~~~",
  "completion": " 1\n"
}
{
  "prompt": "space exploration,space travel ### Do you think humans are the only intelligent life in the universe? ~~~",
  "completion": " 1\n"
}
{
  "prompt": "ecology ### Have natural disasters gotten worse with the increase in human existence? If so, why? ~~~",
  "completion": " 1\n"
}
{
  "prompt": "ice breaker ### What would prompt you to make a dramatic change in your life. Is it worth it? ~~~",
  "completion": " 1\n"
}
{
  "prompt": "philosophy,mathematic ### Why bother with subjecti

In [8]:
!echo "execute: openai tools fine_tunes.prepare_data -f $(pwd)/$out_file_name"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
execute: openai tools fine_tunes.prepare_data -f /Users/louisbeaumont/Documents/langame-worker/notebooks/../data/fine_tune_fitness_classification_01_05_2022.jsonl


In [9]:
train_file = openai.File.create(
  file=open("_prepared_train.jsonl"),
  purpose="fine-tune"
)
valid_file = openai.File.create(
  file=open("_prepared_valid.jsonl"),
  purpose="fine-tune"
)
openai.FineTune.create(
    training_file=train_file["id"],
    validation_file=valid_file["id"],
    model="ada",
    # "On classification tasks, we recommend setting this to false"
    use_packing=False,
    # https://beta.openai.com/docs/api-reference/fine-tunes/create#fine-tunes/create-prompt_loss_weight
    # classification recommended 0.1 (it's the default value anyway)
    prompt_loss_weight=0.1
)

<FineTune fine-tune id=ft-bBksgsOcWJjeZuV9XHiHSvIs at 0x158877a90> JSON: {
  "created_at": 1651376064,
  "events": [
    {
      "created_at": 1651376064,
      "level": "info",
      "message": "Created fine-tune: ft-bBksgsOcWJjeZuV9XHiHSvIs",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.1
  },
  "id": "ft-bBksgsOcWJjeZuV9XHiHSvIs",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-KwcHNgfGe4pqdKDLQIJt99UZ",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 586199,
      "created_at": 1651376061,
      "filename": "_prepared_train.jsonl",
      "id": "file-FjpwIA5Q2FH7afulYnA5Z27f",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "status_details": null
    }
  ],
  "updated_at": 1651376064,
  "validation_files": [
    {
      "bytes": 1

In [10]:
!rm -rf _prepared_train.jsonl _prepared_valid.jsonl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Custom model topic inference

In [2]:

logger = logging.getLogger("classification")
memes, index, embeddings_model = get_existing_conversation_starters(
    c._firestore_client, logger=logger, confirmed=True, push_to_hub=True)

Skipping id: 4XlXSjPhH6QSGZDzwIQe, garbage
Skipping id: 4sIjdMJr0exliT9yfvuL, garbage
Skipping id: 6jNOw0e7VA6lHWddtqMy, garbage
Skipping id: 9ZfEEkrB3cjLooW6YZM2, garbage
Skipping id: B8hRGbGGOweqEyxuuplE, garbage
Skipping id: CYsOWtp1ed2XDeplB5O5, garbage
Skipping id: DlXglThZUiPJcuyLLKHV, garbage
Skipping id: EBZCFk0aZPVS1SinJwcz, garbage
Skipping id: FP0UBU9PBzttN2d7YFvZ, garbage
Skipping id: ISOBHbvYDxi04VEFPZWw, garbage
Skipping id: Iu1De6gLJ0z7eoi7Qb96, garbage
Skipping id: NtXdp7ODn4sR3YR2eEW6, garbage
Skipping id: P05T4keIh8bvF1SKVSSw, garbage
Skipping id: RCI4BcA6Bf1kYBk7hUTe, garbage
Skipping id: U0DUXOSpfqYsgurOyL3L, garbage
Skipping id: Uikgs6eO85MRivL0peL6, garbage
Skipping id: XIPyRwJloK6ZdAdTHOeu, garbage
Skipping id: XdaHW0tJ9wIGAos7WoG4, garbage
Skipping id: Y2V1ZLMVggTj45u8RmzK, garbage
Skipping id: bCW1uPMxDEy6iO3oMRQq, garbage
Skipping id: bdf0W2iG9B9rZXbe4xun, garbage
Skipping id: d9FxG5haHnN2ku7spAng, garbage
Skipping id: eyzSmD1AdfKkwOYtfoof, garbage
Skipping id

ArrowInvalid: Column 1 named validation expected length 1956 but got length 490

In [5]:

out_file_name = f"../data/fine_tune_topic_classification_{datetime.date.today().strftime('%d_%m_%Y')}.txt"

# make a dict of {"TOPIC": int}
topic_to_int = {}
for e in memes:
    for topic in e['topics']:
        if topic not in topic_to_int:
            topic_to_int[topic] = len(topic_to_int)
try:
    os.remove(out_file_name)
except:
    pass
for e in memes:
    with open(out_file_name, "a+") as outfile:
        if len(e['topics']) == 0: continue
        outfile.write(f"{e['content']} ### {','.join(map(str, e['topics']))}\n")

In [8]:
!head $out_file_name -n3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
When is a time when you know for sure you'll soon have to exchange a nice, meaningful conversation with someone new? ### ice breaker
Do you think humans are the only intelligent life in the universe? ### space exploration,space travel
Have natural disasters gotten worse with the increase in human existence? If so, why? ### ecology


In [6]:

# split the file into train.txt and eval.txt
with open(out_file_name, "r") as f:
    lines = f.readlines()
    train_lines = lines[:int(len(lines) * 0.8)]
    eval_lines = lines[int(len(lines) * 0.8):]
    with open("train.txt", "w") as f:
        f.writelines(train_lines)
    with open("eval.txt", "w") as f:
        f.writelines(eval_lines)
dataset = load_dataset('text', data_files={"train": "train.txt", "eval": "eval.txt"})

Using custom data configuration default-f18bc8eb4d57408a


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /home/gpu/.cache/huggingface/datasets/text/default-f18bc8eb4d57408a/0.0.0/acc32f2f2ef863c93c2f30c52f7df6cc9053a1c2230b8d7da0d210404683ca08. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [37]:

model = AutoModelForCausalLM.from_pretrained("distilgpt2")
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
# tokenizer.eos_token = 198
# tokenizer.pad_token = tokenizer.eos_token


def preprocess_function(x):
    return tokenizer([e + "\n" for e in x["text"]])
tokenized_eli5 = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=["text"],
)
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result
lm_datasets = tokenized_eli5.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

loading configuration file https://huggingface.co/distilgpt2/resolve/main/config.json from cache at /home/gpu/.cache/huggingface/transformers/f985248d2791fcff97732e4ee263617adec1edb5429a2b8421734c6d14e39bee.422318838d1ec4e061efb4ea29671cb2a044e244dc69229682bebd7cacc81631
Model config GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summa

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [51]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    # learning_rate=2e-5,
    # weight_decay=0.01,
    num_train_epochs=2,
    # auto_find_batch_size=True,
    # push_to_hub_model_id="distilgpt2-starter-classification",
    # push_to_hub_organization="Langame",
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["eval"],
    # data_collator=data_collator,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/home/gpu/langame-worker/env/lib/python3.8/site-packages/transformers/training_args.py:1186: FutureWarning: `--push_to_hub_model_id` and `--push_to_hub_organization` are deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case Langame/distilgpt2-starter-classification).
  warnings.warn(


OSError: Tried to clone a repository in a non-empty folder that isn't a git repository. If you really want to do this, do it manually:
git init && git remote add origin && git pull origin main
 or clone repo to a new folder and move your existing files there afterwards.

In [39]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 100
  Batch size = 8


Perplexity: 197.97


In [40]:
import torch
starter = "If an artificial intelligence were to request permission to exist, what would your decision be? ###"
input_ids = torch.tensor([tokenizer.encode(starter)]).to("cuda")
beam_output = model.generate(
    input_ids,  
    max_length=50, 
    num_beams=5, 
    early_stopping=True,
    eos_token_id=198,
)
tokenizer.decode(beam_output[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.


'If an artificial intelligence were to request permission to exist, what would your decision be? ### artificial intelligence,ai,ai alignment\n'

In [41]:
tokenizer.save_pretrained("local-pt-checkpoint")
model.save_pretrained("local-pt-checkpoint")

tokenizer config file saved in local-pt-checkpoint/tokenizer_config.json
Special tokens file saved in local-pt-checkpoint/special_tokens_map.json
Configuration saved in local-pt-checkpoint/config.json
Model weights saved in local-pt-checkpoint/pytorch_model.bin


In [43]:
!python -m transformers.onnx --model=local-pt-checkpoint onnx/ --feature=causal-lm

Using framework PyTorch: 1.12.0+cu116
Overriding 1 configuration item(s)
	- use_cache -> False
/home/gpu/langame-worker/env/lib/python3.8/site-packages/transformers/models/gpt2/modeling_gpt2.py:798: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if batch_size <= 0:
Validating ONNX model...
	-[✓] ONNX model output names match reference model ({'logits'})
	- Validating ONNX Model output "logits":
		-[✓] (2, 8, 50257) matches (2, 8, 50257)
		-[✓] all values close (atol: 1e-05)
All good, model saved at: onnx/model.onnx


In [47]:
!mv onnx/model.onnx distilgpt2-starter-classification

In [49]:
trainer.push_to_hub("distilgpt2-starter-classification", private=True, organization="Langame")

OSError: Tried to clone a repository in a non-empty folder that isn't a git repository. If you really want to do this, do it manually:
git init && git remote add origin && git pull origin main
 or clone repo to a new folder and move your existing files there afterwards.

In [13]:
tokenizer = AutoTokenizer.from_pretrained("Langame/distilgpt2-starter-classification", use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained("Langame/distilgpt2-starter-classification", use_auth_token=True)
tokenizer.eos_token_id = 198
tokenizer.pad_token_id = tokenizer.eos_token_id
device = "cuda:0" if True else "cpu"
if device == "cuda:0":
    model.cuda()
import torch
starter = "If an artificial intelligence were to request permission to exist, what would your decision be? ###"
input_ids = torch.tensor([tokenizer.encode(starter)]).to("cuda")
beam_output = model.generate(
    input_ids,  
    return_dict_in_generate=True,
    eos_token_id=198,  # line break
    max_length=50,
    num_return_sequences=1,
    return_text=False,
    return_full_text=False,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    device=device,
)

topics = tokenizer.decode(beam_output["sequences"].tolist()[0]).replace(
    starter, ""
).strip()  # TODO: return_text doesn't work for some reason
list(set(topics.split(",")))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:198 for open-end generation.


['artificial intelligence', 'ai alignment', 'ai']

In [18]:
import requests
API_KEY = "YOUR_API_KEY"
API_URL = "https://api-inference.huggingface.co/models/Langame/distilgpt2-starter-classification"
headers = {"Authorization": f"Bearer {API_KEY}"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

output = query({
	"inputs": starter,
	"parameters": {
		"max_length": 50,
		"num_return_sequences": 1,
		"return_text": False,
		"return_full_text": False,
		"do_sample": True,
		"top_k": 50,
		"top_p": 0.95,
		"end_sequence": "\n",
	},
	"options": {
		"wait_for_model": True,
		"use_cache": True,  # TODO: should be in public api args
	},
})
output

[{'generated_text': ' history,ai,artificial intelligence\n'}]